**Scratchpad**

**Imports**

In [ ]:
%matplotlib inline

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import models, transforms, datasets

import os

**Get Data from Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!cp /content/gdrive/'My Drive'/pytorch_challenge/flower_data.zip /content
!unzip /content/flower_data.zip
# !ls /content/flower_data

**Downloads**

In [ ]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
#accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
accelerator = 'cu80'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch

In [ ]:
!pip install --no-cache-dir -I pillow

**Constants/Hyperparameters**

In [ ]:
TRN_DATA_DIR = "/content/flower_data/train"
VAL_DATA_DIR = "/content/flower_data/valid"
BATCH_SIZE = 64
NUM_EPOCHS = 20
LOAD_MODEL = True

# 'inception', 'resnet18', 'resnet152',
# 'densenet121', 'densenet201'
MODEL_TO_USE = 'densenet201'
INPUT_SIZE = { 'inception': (299, 299, 3),
               'resnet': (224, 224, 3),
               'densenet': (224, 224, 3) }
OUTPUT_SIZE = 102
MODEL_DIR = 'checkpoints'
MODEL_FN = 'densenet201_20e.pth'

In [ ]:
!mkdir 'checkpoints'
!cp /content/gdrive/'My Drive'/pytorch_challenge/checkpoints/densenet201_20e.pth checkpoints/
!ls checkpoints

**Utility Functions**

In [ ]:
def get_data_gen(input_size):
    '''Initialize data loader
    '''
    side_len = min(input_size[:2])
    # for training
    train_transforms = transforms.Compose([transforms.Resize(side_len),
                                           transforms.RandomCrop(side_len),
                                           transforms.RandomHorizontalFlip(),
                                           transforms.RandomRotation(30),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                std=[0.229, 0.224, 0.225])])

    train_data = datasets.ImageFolder(TRN_DATA_DIR, transform=train_transforms)
    trainloader = torch.utils.data.DataLoader(train_data,
                                              batch_size=BATCH_SIZE, shuffle=True)

    # for validation
    valid_transforms = transforms.Compose([transforms.Resize(side_len),
                                           transforms.RandomCrop(side_len),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                std=[0.229, 0.224, 0.225])])

    valid_data = datasets.ImageFolder(VAL_DATA_DIR, transform=valid_transforms)
    validloader = torch.utils.data.DataLoader(valid_data,
                                              batch_size=BATCH_SIZE, shuffle=True)

    return trainloader, validloader

def get_modified_squeezenet():
    '''Return squeezenet with frozen layers
    '''
    model = models.squeezenet1_1(pretrained=True)

    # Squeezenet contains two Sequential
    seqs = list(model.children())
    # first Sequential is for feature extraction
    for param in seqs[0].parameters():
        param.requires_grad = False

    # second Sequential is for classification
    # we replace only the output size, which is the
    # output channel of the Conv2d
    for child in seqs[1]:
        if child.__class__.__name__ == "Conv2d":
            # replace Conv2d
            print("replace")

    return model 

def get_modified_inception():
    '''Return inception3 with frozen layers
    '''
    model = models.inception_v3(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False

    # replace fc
    classifier = nn.Sequential(
        nn.Linear(2048, 1024),
        nn.ReLU(),
        nn.Dropout(p=0.6),
        nn.Linear(1024, OUTPUT_SIZE)
    )

    model.fc = classifier
    # work around to remove aux from output
    model.aux_logits = False

    return model 
  
def get_modified_resnet(size):
    '''Return resnet with frozen layers
    '''
    if size == 18:
        model = models.resnet18(pretrained=True)
        in_features = 512
    elif size == 152:
        model = models.resnet152(pretrained=True)
        in_features = 2048
    else:
        raise Exception("Unsupported model type!")

    for param in model.parameters():
        param.requires_grad = False

    # replace fc
    classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(p=0.6),
            nn.Linear(512, OUTPUT_SIZE)
        )

    model.fc = classifier

    return model

def get_modified_densenet(size, mode='freeze_dense'):
    '''Return densenet with frozen layers
    '''
    if size == 121:
        model = models.densenet121(pretrained=True)
        in_features = 1024
    elif size == 201:
        model = models.densenet201(pretrained=True)
        in_features = 1920
    else:
        raise Exception("Unsupported model type!")

    for param in model.parameters():
        param.requires_grad = False

    # replace classifier
    classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(p=0.6),
            nn.Linear(512, OUTPUT_SIZE)
        )

    model.classifier = classifier
    
    if mode == 'freeze_linear':
            
        for param in model.features.denseblock4.parameters():
            param.requires_grad = True                   
            
        for param in model.classifier.parameters():
            param.requires_grad = False

    return model  

**Train**

In [ ]:
def check_accuracy(model, validloader, loss_f, train_on_gpu):
    with torch.no_grad():
        total = 0
        corrects = 0
        val_loss = 0
        model.eval()
        for images, labels in tqdm(validloader):
            # move data to gpu
            if(train_on_gpu):
                images, labels = images.cuda(), labels.cuda()

            logits = model.forward(images)
            val_loss += loss_f(logits, labels)

            probs = F.softmax(logits, dim=1)
            preds = probs.cpu().numpy().argmax(axis=1)
            preds = torch.from_numpy(preds)
            if(train_on_gpu):
                preds = preds.cuda()
            corrects += torch.sum(preds == labels).type(torch.FloatTensor)
            total += len(labels)

    accuracy = float(corrects / total)
    print("Validation Accuracy: {:.3f}".format(accuracy))

In [ ]:
def train(model_name, model):
    '''Perform model training
    '''
    # prepare data generator
    input_size = INPUT_SIZE[model_name]
    trainloader, validloader = get_data_gen(input_size)

    if LOAD_MODEL:
        ckpt = torch.load(os.path.join(MODEL_DIR, MODEL_FN))
        model.load_state_dict(ckpt)
        #print("Loaded %s." % (os.path.join(MODEL_DIR, MODEL_FN)))

    # First checking if GPU is available
    train_on_gpu=torch.cuda.is_available()
    if(train_on_gpu):
        print('Training on GPU.')
        model.cuda()
    else:
        print('No GPU available, training on CPU.')

    # prepare loss and optimizer functions
    loss_f = nn.CrossEntropyLoss()
    #optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

    optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                          lr=0.01, momentum=0.9)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                     factor=0.1, patience=1,
                                                     verbose=True)    
    
    if LOAD_MODEL:
       check_accuracy(model, validloader, loss_f, train_on_gpu)
    
    # perform training
    train_losses, val_losses = [], []
    best_val_loss = np.inf
    for e in range(NUM_EPOCHS):
        running_loss = 0
        model.train()
        for images, labels in tqdm(trainloader):
            # move data to gpu
            if(train_on_gpu):
                images, labels = images.cuda(), labels.cuda()
            # Clear the gradients, do this because gradients are accumulated
            optimizer.zero_grad()
            # Forward pass, then backward pass, then update weights
            logits = model.forward(images)
            loss = loss_f(logits, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        else:
            with torch.no_grad():
                total = 0
                corrects = 0
                val_loss = 0
                model.eval()
                for images, labels in tqdm(validloader):
                    # move data to gpu
                    if(train_on_gpu):
                        images, labels = images.cuda(), labels.cuda()

                    logits = model.forward(images)
                    val_loss += loss_f(logits, labels)

                    probs = F.softmax(logits, dim=1)
                    preds = probs.cpu().numpy().argmax(axis=1)
                    preds = torch.from_numpy(preds)
                    if(train_on_gpu):
                        preds = preds.cuda()
                    corrects += torch.sum(preds == labels).type(torch.FloatTensor)
                    total += len(labels)

            accuracy = float(corrects / total)
            train_losses.append(running_loss/len(trainloader))
            val_losses.append(val_loss/len(validloader))

            print("Epoch: {}/{}.. ".format(e+1, NUM_EPOCHS),
                  "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
                  "Validation Loss: {:.3f}.. ".format(val_loss/len(validloader)),
                  "Validation Accuracy: {:.3f}".format(accuracy))

            print(f"Training loss: {running_loss/len(trainloader)}")

            if val_loss < best_val_loss:
                # save model
                torch.save(model.state_dict(), os.path.join(MODEL_DIR, MODEL_FN))
                print("Saved to %s." % os.path.join(MODEL_DIR, MODEL_FN))
                best_val_loss = val_loss
            
            # adjust learning rate based on validation loss
            scheduler.step(val_loss)

    # graph training and validation loss
    plt.plot(train_losses, label='Training loss')
    plt.plot(val_losses, label='Validation loss')
    plt.legend(frameon=False)
    plt.show()

**Execution**

In [ ]:
MODEL_TO_USE = 'densenet201'
if MODEL_TO_USE == 'squeezenet':
    model = get_modified_squeezenet()
elif MODEL_TO_USE == 'inception':
    model = get_modified_inception()
elif 'resnet' in MODEL_TO_USE:
    sz = int(MODEL_TO_USE.strip('resnet'))
    model = get_modified_resnet(sz)
    MODEL_TO_USE = 'resnet'
elif 'densenet' in MODEL_TO_USE:
    sz = int(MODEL_TO_USE.strip('densenet'))
    model = get_modified_densenet(sz, mode='freeze_linear')
    MODEL_TO_USE = 'densenet'
else:
    print("Unsupported model type!")

train(MODEL_TO_USE, model)
!cp checkpoints/densenet201_20e.pth /content/gdrive/'My Drive'/pytorch_challenge/checkpoints